# Check that we are correctly solving for the steady-state
When $m < N-f$ for both receptor types, solution is fully analytical, unique, etc. No issue there. 

When both receptor types have $m >= N-f$, it's a 2D system of equations for $I_T$ and $I_C$

In [1]:
import numpy as np
import scipy as sp
from scipy.optimize import root as rootsolve, root_scalar
import matplotlib.pyplot as plt
import seaborn as sns

# Local modules
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

from models.akpr_i_model import solution_CT
from models.tcr_car_akpr_model import psi_of_i_gamma

## Select model parameters


In [2]:
def wrap_param_choices(tcr_params, tcr_nmf, car_fits, car_k, car_nmf): 
    tcr_car_rates = [
        [tcr_params[0], tcr_params[0]*0.1], 
        [tcr_params[1], tcr_params[1]*10.0], 
        [tcr_params[2], car_fits[0]], 
        [tcr_params[3], car_fits[1]],
        [tcr_params[4], car_k], 
        [[1.0, car_fits[2]], [car_fits[3], 1.0]], 
        [tcr_params[5], car_fits[4]]
    ]
    tcr_car_rates = list(map(np.asarray, tcr_car_rates))

    tcr_car_nmf = zip(tcr_nmf, car_nmf)
    tcr_car_nmf = list(map(np.asarray, tcr_car_nmf))
    return tcr_car_rates, tcr_car_nmf

In [3]:
# Compute a test model panel
def check_model_output_tcr_car(model, rates, ri_tots, nparams, cd19_l_tau, 
                               n_tau=101, tcr_l_range=None, tcr_tau_range=None):
    """ horizontal range will be tau_tcr (axis 1 of output array)
    vertical range will be L_tcr
    The CAR quantities will be held constant. """
    # Define the range of L_i, tau_i to test
    if tcr_tau_range is None:
        tcr_tau_range = np.linspace(0.001, 10.001, n_tau)
    if tcr_l_range is None:
        tcr_l_range = np.asarray([650.0, 6500.0, 65000.0])  # 1nM, 10 nM, 1 uM
    # Index each output array [l2, tau2]
    outputs = np.zeros([2, tcr_l_range.size, tcr_tau_range.size])  # TCR, CAR outputs
    for i in range(outputs.shape[1]):  # over L_tcr
        lvec = np.asarray([tcr_l_range[i], cd19_l_tau[0]])
        for j in range(outputs.shape[2]):  # over tau_tcr
            tauvec = np.asarray([tcr_tau_range[j], cd19_l_tau[1]])
            lvec = np.asarray([tcr_l_range[i], cd19_l_tau[0]])
            complexes = model(rates, tauvec, lvec, ri_tots, nparams)
            outputs[0, i, j] = complexes[0][-1]
            outputs[1, i, j] = complexes[1][-1]
            
    return tcr_l_range, tcr_tau_range, outputs

In [4]:
def plot_model_output(outputs, lrange, taurange, lstyle="-", lw=2.0, palette_i=None, figaxes=None):
    # Color for each L. Outputs shaped [receptor, L, tau]
    if palette_i is None:
        palette_i = sns.color_palette(n_colors=outputs.shape[1])
    if figaxes is None:
        fig, axes = plt.subplots(1, outputs.shape[0])
        fig.set_size_inches(4.0*outputs.shape[0], 3.5)
    else:  # to plot on top of existing stuff
        fig, axes = figaxes
    axes = axes.flatten()
    for i in range(outputs.shape[1]):
        axes[0].plot(taurange, outputs[0, i], label=lrange[i], ls=lstyle, color=palette_i[i], lw=lw)
        axes[1].plot(taurange, outputs[1, i], label=lrange[i], ls=lstyle, color=palette_i[i], lw=lw)
    return [fig, axes]

In [5]:
## Select unchanging parameters
# Choose tau, and use experimentally calibrated L for CD19
choice_cd19_l_tau = (1e5, 50.0)
tcr_car_ritots = [np.asarray([9e4, 1e6]), 1.0]


# Example of the changing params. We change TCR, CAR params follow suit. 
# Easy parameters
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 1132.8, 3.28e-2, 1, 4.29e-3],
    tcr_nmf = [6, 2, 2],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct, psi0
    car_fits = [10000.0, 1e-2, 100.0, 100.0, 4.29e-3],
    car_k = 1,
    car_nmf = [3, 1, 2]  # Easy yet implicit CAR
)

## Original solution method
2D equation as soon as one receptor type is implicit. 
Doesn't work too well. 

In [ ]:
## Solving for SHP-1 activated by different receptor types.
# For cases where at least one type has m >= N-f
def equation_svec_types_brute(svec, ratesp, tausp, ctotsols, stot, nparams):
    """ Equation to solve for \vec{S} """
    phis, kappas, cmthreshs, sthreshs, ks, gamma_mat, psi0s = ratesp
    n_types = len(svec)
    ns, ms, fs = nparams

    # Compute psi(S) of each type, given the tentative svec
    # and the 1d arrays of parameters phis, psi0s, etc.
    psis = psi_of_i_gamma(svec, sthreshs, ks, phis, gamma_mat, psi0s)

    # Compute the C_m of each type. Hard because m, n, f may differ.
    cmvec = np.zeros(n_types)
    for i in range(n_types):
        geofact = phis[i]*tausp[i] / (1.0 + phis[i]*tausp[i])
        if ms[i] < ns[i] - fs[i] or fs[i] == 0:  # feedforward case
            cmvec[i] = ctotsols[i] / (phis[i]*tausp[i]+ 1.0) * geofact**ms[i]
        elif ms[i] == ns[i] - fs[i]:  # Intermediate
            cmvec[i] = ctotsols[i] / (psis[i]*tausp[i]+ 1.0) * geofact**ms[i]
        elif ms[i] < ns[i]:  # Somewhere in the chain of complexes affected by psi(S)
            cmvec[i] = ctotsols[i] / (psis[i]*tausp[i]+ 1.0) * geofact**(ns[i] - fs[i])
            cmvec[i] *= (psis[i]*tausp[i] / (psis[i]*tausp[i] + 1.0))**(ms[i] - ns[i] + fs[i])
        elif ms[i] == ns[i]:  # Last complex
            cmvec[i] = ctotsols[i] * geofact**(ns[i] - fs[i])
            cmvec[i] *= (psis[i]*tausp[i] / (psis[i]*tausp[i] + 1.0))**fs[i]
        else:
            raise ValueError(("N = {}, m = {}, f = {}"
                + " for receptor type {}").format(ns[i], ms[i], fs[i], i))

    # Compute the RHS, function of S = S
    cmvec = cmvec / cmthreshs  # C_m / C_thresh
    svec_new = stot * cmvec / (1.0 + np.sum(cmvec))
    return svec - svec_new  # Should be equal to zero


# Main functions solving the SHP-1 model with multiple receptor types.
def steady_akpr_i_receptor_types_brute(ratesp, tausp, lsp, ri_tots, nparams):
    """Solving for the steady_state of the SHP-1 model with two
    different receptors, each with its own ligand.
    So, we need to solve for S_tot, C_tot and D_tot first.

    Args:
        ratesp (list of np.ndarrays): [phi_arr, kappa_arr, cmthresh, sthresh_arr,
            k_arr, gamma_mat, psi_arr] where x_arr is
            a 1d array of parameter x values for each receptor type.
            except cmthresh which is unique,
            and gammat is a KxK array for K receptor types,
            containing \gamma_{ij}, contribution of SHP-1 bound to receptor type j
            to the negative feedforward on receptor type i.
        tausp (np.ndarray of floats): binding time of the ligands
            of each type of receptor
        lsp (np.ndarray of floats): total number of ligands of each type
        ri_tots (list of 1 ndarray, 1 float):
            Rsp (np.ndarray of floats): total number of receptors of each type
            STp (float): total number of SHP-1 molecules
        nparams (list of 3 np.ndarrays): Ns, ms, fs of all receptor types.

    Returns:
        complexes (np.ndarray): list of 1D arrays of complexes or S, ordered
            [ [C_0, C_1, ..., C_N],
              [D_0, D_1, ..., D_N'],
              [S_1, S_2]
            ]
    """
    # Exploit the vector-compatible form of the function solving for C_Ts
    phis, kappas, cmthreshs, sthreshs, ks, gamma_mat, psi0s  = ratesp
    Rsp, STp = ri_tots
    n_types = len(Rsp)
    # C_n for each type, and lastly the S vector.
    complexes = [np.zeros(n+1) for n in nparams[0]] + [np.zeros(n_types)]

    # Special case: no input
    if np.sum(lsp) == 0.0 or np.sum(tausp) == 0.0:
        return complexes

    # Solve for C_Ts
    C_tot_sols = solution_CT(lsp, tausp, kappas, Rsp)
    ns, ms, fs = nparams

    # Quantities that come back often
    geofacts = phis*tausp / (phis*tausp + 1.0)

    # Compute all Cns and Dns below N-f: apply same factor recursively
    for i in range(n_types):
        if ns[i] == fs[i]: continue  # No complex to compute
        # Otherwise, we can at least compute C_0
        complexes[i][0] = C_tot_sols[i] / (phis[i]*tausp[i] + 1.0)
        for n in range(1, ns[i]-fs[i]):
            complexes[i][n] = complexes[i][n-1]*geofacts[i]

    # Now, compute the vector S and psi(S)
    # Case where all m < N - f
    if sum([ms[i] >= ns[i]-fs[i] for i in range(n_types)]) == 0:
        cmnorm_vec = np.asarray([complexes[i][ms[i]]/cmthreshs[i] for i in range(n_types)])
        sum_cm = np.sum(cmnorm_vec)
        svec = STp * cmnorm_vec / (1.0 + sum_cm)
    else:
        initial_s_guess = 0.5 * STp * (C_tot_sols / (1.0 + np.sum(C_tot_sols)))
        sol = rootsolve(equation_svec_types_brute, x0=initial_s_guess, tol=1e-6,
                        args=(ratesp, tausp, C_tot_sols, STp, nparams))
        if not sol.success:
            raise RuntimeError("Could not solve for S. "
                + "Error message: {}".format(sol.message))
        else:
            svec = sol.x

    if np.sum(svec) > STp:
        raise ValueError("Found a I solution going over the max number of I")

    # Compute the psi(S) vector with the solution svec.
    psis = psi_of_i_gamma(svec, sthreshs, ks, phis, gamma_mat, psi0s)

    # Finally, compute C_{N-f}, ..., C_N for each receptor type
    for i in range(n_types):
        n = ns[i] - fs[i]
        if n > 0:
            complexes[i][n] = complexes[i][n-1] * phis[i]*tausp[i] / (psis[i]*tausp[i] + 1.0)
        else:
            complexes[i][n] = C_tot_sols[i] / (psis[i]*tausp[i] + 1.0)

        geofact = psis[i]*tausp[i] / (psis[i]*tausp[i] + 1.0)
        for n in range(ns[i] - fs[i] + 1, ns[i]):
            complexes[i][n] = complexes[i][n-1] * geofact
        complexes[i][ns[i]] = complexes[i][ns[i]-1] * psis[i]*tausp[i]

    # Add the vector of S values to the returned variables
    complexes[-1] = svec

    # Return
    return complexes

In [ ]:
# Easy parameters
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 1132.8, 3.28e-2, 1, 4.29e-3],
    tcr_nmf = [6, 2, 2],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct, psi0
    car_fits = [1000.0, 1e-1, 10.0, 100.0, 4.29e-3],
    car_k = 1,
    car_nmf = [3, 1, 2]  # Easy yet implicit CAR
)

l_range_brute, tau_range_brute, brute_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_brute, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

figaxes = plot_model_output(brute_outs, l_range_brute, tau_range_brute, lstyle="-")
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

In [ ]:
# Hard parameters
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 7e3, 4e-5, 1, 0.0],
    tcr_nmf = [6, 4, 1],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct, psi0
    #car_fits = [3000.0, 10**(-3.5), 0.2, 0.02],
    car_fits = [1000.0, 1e-2, 10.0, 10.0, 0.0],
    car_k = 1,
    car_nmf = [3, 2, 3]  # Hard implicit CAR
)
# A log10 car fits causing RuntimeError issues
# 3.47425152 -3.56416617 -0.89229255 -1.85015833  1.09313421  1.15144979

l_range_brute, tau_range_brute, brute_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_brute, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

figaxes = plot_model_output(brute_outs, l_range_brute, tau_range_brute, lstyle="-")
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

## Better conditioned method
Solve for $T_m$ or $C_m$ instead of $\vec{S}$ when one receptor type is implicit. 

In [ ]:
def equation_one_cm_types(cmii, ii, cmnorms, ratesp, tausp, C_tot_sols, stot, nparams):
    """ From a guess cm for receptor type ii, when other C_m^i are known and
    stored in cmnorms, compute the S vector and from there C_m^i to establish an equation. 
    """
    # Extract parameters of receptor type ii
    phip, kappap, cmthresh, sthresh, kp, gamma_mati, psi0p  = [r[ii] for r in ratesp]
    c_tot_i = C_tot_sols[ii]
    n_p, mp, fp = [r[ii] for r in nparams]
    
    # Replace in cmnorms with the guess cm
    cmthreshs = ratesp[2]
    cmnorms[ii] = cmii / cmthreshs[ii]
    
    # Compute vector of S and psi_s from there
    sum_cmnorms = np.sum(cmnorms)
    svec = stot * cmnorms / (1.0 + sum_cmnorms)
    # sthresh, ks, phip, gammat, psi0
    psivec = psi_of_i_gamma(svec, *ratesp[3:5], ratesp[0], *ratesp[5:])
    
    # Recompute C_m from there
    geofact = phip*tausp[ii] / (phip*tausp[ii] + 1.0)

    # Compute all Cns and Dns below N-f: apply same factor recursively
    # Assuming m_i >= N_i - f_i so we can go to m=N-f at least
    cmii_eq = c_tot_i * geofact**(n_p - fp)
    geofact = psivec[ii]*tausp[ii] / (psivec[ii]*tausp[ii] + 1.0)
    if n_p > mp >= n_p - fp:
        cmii_eq *= geofact**(mp - n_p + fp) / (psivec[ii]*tausp[ii] + 1.0)
    elif mp == n_p:
        cmii_eq *= geofact**fp
    return cmii_eq - cmii
    

# Main functions solving the SHP-1 model with multiple receptor types.
def steady_akpr_i_receptor_types_better(ratesp, tausp, lsp, ri_tots, nparams):
    """Solving for the steady_state of the SHP-1 model with two
    different receptors, each with its own ligand.
    So, we need to solve for S_tot, C_tot and D_tot first.

    Args:
        ratesp (list of np.ndarrays): [phi_arr, kappa_arr, cmthresh, sthresh_arr,
            k_arr, gamma_mat, psi_arr] where x_arr is
            a 1d array of parameter x values for each receptor type.
            except cmthresh which is unique,
            and gammat is a KxK array for K receptor types,
            containing \gamma_{ij}, contribution of SHP-1 bound to receptor type j
            to the negative feedforward on receptor type i.
        tausp (np.ndarray of floats): binding time of the ligands
            of each type of receptor
        lsp (np.ndarray of floats): total number of ligands of each type
        ri_tots (list of 1 ndarray, 1 float):
            Rsp (np.ndarray of floats): total number of receptors of each type
            STp (float): total number of SHP-1 molecules
        nparams (list of 3 np.ndarrays): Ns, ms, fs of all receptor types.

    Returns:
        complexes (np.ndarray): list of 1D arrays of complexes or S, ordered
            [ [C_0, C_1, ..., C_N],
              [D_0, D_1, ..., D_N'],
              [S_1, S_2]
            ]
    """
    # Exploit the vector-compatible form of the function solving for C_Ts
    phis, kappas, cmthreshs, sthreshs, ks, gamma_mat, psi0s  = ratesp
    Rsp, STp = ri_tots
    n_types = len(Rsp)
    # C_n for each type, and lastly the S vector.
    complexes = [np.zeros(n+1) for n in nparams[0]] + [np.zeros(n_types)]

    # Special case: no input
    if np.sum(lsp) == 0.0 or np.sum(tausp) == 0.0:
        return complexes

    # Solve for C_Ts
    C_tot_sols = solution_CT(lsp, tausp, kappas, Rsp)
    ns, ms, fs = nparams

    # Quantities that come back often
    geofacts = phis*tausp / (phis*tausp + 1.0)

    # Compute all Cns and Dns below N-f: apply same factor recursively
    for i in range(n_types):
        if ns[i] == fs[i]: continue  # No complex to compute
        # Otherwise, we can at least compute C_0
        complexes[i][0] = C_tot_sols[i] / (phis[i]*tausp[i] + 1.0)
        for n in range(1, ns[i]-fs[i]):
            complexes[i][n] = complexes[i][n-1]*geofacts[i]

    # Now, compute the vector S and psi(S)
    # Case where all m < N - f
    number_implicit = sum([ms[i] >= ns[i]-fs[i] for i in range(n_types)]) 
    if number_implicit == 0:
        cmnorm_vec = np.asarray([complexes[i][ms[i]]/cmthreshs[i] for i in range(n_types)])
        sum_cm = np.sum(cmnorm_vec)
        svec = STp * cmnorm_vec / (1.0 + sum_cm)
    
    elif number_implicit == 1:
        #print("ratesp:", ratesp, "\ntausp:", tausp, "\nlsp:", lsp,
        #            "\nri_tots:", ri_tots, "\nnparams:", nparams)
        # Only one receptor type is implicit; can reduce to a scalar equation 
        # for the C_m of that type. 
        impli = np.argmax(ms >= ns - fs)
        # Get a vector of the explicit known Cms, we don't recompute every time. 
        # The unknown cm element will be 0 anyways
        cmnorm_vec = np.asarray([complexes[i][ms[i]]/cmthreshs[i] for i in range(n_types)])
        # Certainly, C_m^i can't be more than R^i minus the sum of already computed C_n^i
        bracket = (0.0, Rsp[impli])# - np.sum(complexes[impli]))
        args = (impli, cmnorm_vec, ratesp, tausp, C_tot_sols, STp, nparams)
        sol = root_scalar(equation_one_cm_types, x0=np.mean(bracket), 
                        args=args, bracket=bracket)
        if not sol.converged:
            raise RuntimeError("Could not solve for S. "
                + "Error flag: {}".format(sol.flag))
        else:
            # From the missing cm_i, compute the vector of S
            # Check that the found cm_i is consistent later
            cm_impli = sol.root
            cmnorm_vec[impli] = cm_impli / cmthreshs[impli]
            sum_cm = np.sum(cmnorm_vec)
            svec = STp * cmnorm_vec / (1.0 + sum_cm)
        
        
    else: # Two or more receptor types have a true feedback
        initial_s_guess = 0.5 * STp * (C_tot_sols / (1.0 + np.sum(C_tot_sols)))
        sol = rootsolve(equation_svec_types_brute, x0=initial_s_guess, tol=1e-6,
                        args=(ratesp, tausp, C_tot_sols, STp, nparams))
        if not sol.success:
            raise RuntimeError("Could not solve for S. "
                + "Error message: {}".format(sol.message))
        else:
            svec = sol.x

    if np.sum(svec) > STp:
        raise ValueError("Found a I solution going over the max number of I")

    # Compute the psi(S) vector with the solution svec.
    psis = psi_of_i_gamma(svec, sthreshs, ks, phis, gamma_mat, psi0s)

    # Finally, compute C_{N-f}, ..., C_N for each receptor type
    for i in range(n_types):
        n = ns[i] - fs[i]
        if n > 0:
            complexes[i][n] = complexes[i][n-1] * phis[i]*tausp[i] / (psis[i]*tausp[i] + 1.0)
        else:
            complexes[i][n] = C_tot_sols[i] / (psis[i]*tausp[i] + 1.0)

        geofact = psis[i]*tausp[i] / (psis[i]*tausp[i] + 1.0)
        for n in range(ns[i] - fs[i] + 1, ns[i]):
            complexes[i][n] = complexes[i][n-1] * geofact
        complexes[i][ns[i]] = complexes[i][ns[i]-1] * psis[i]*tausp[i]
    # Check consistency
    if number_implicit == 1:
        if abs(cm_impli - complexes[impli][ms[impli]]) > 1e-6:
            print("Difference in solution and final computed C_m:", cm_impli, complexes[impli][ms[impli]])
            print("ratesp:", ratesp, "\ntausp:", tausp, "\nlsp:", lsp,
                    "\nri_tots:", ri_tots, "\nnparams:", nparams)
    # Add the vector of S values to the returned variables
    complexes[-1] = svec

    # Return
    return complexes

In [ ]:
# Test the same easy condition as with the original method to confirm the new one is OK too
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 1132.8, 3.28e-2, 1, 4.29e-3],
    tcr_nmf = [6, 2, 2],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct, psi0
    car_fits = [1000.0, 1e-1, 10.0, 100.0, 4.29e-3],
    car_k = 1,
    car_nmf = [3, 1, 2]  # Easy yet implicit CAR
)

l_range_better, tau_range_better, better_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_better, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)
l_range_brute, tau_range_brute, brute_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_brute, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

figaxes = plot_model_output(better_outs, l_range_better, tau_range_better, lstyle="-")
figaxes = plot_model_output(brute_outs, l_range_brute, tau_range_brute, lstyle="--", lw=3.0, figaxes=figaxes)
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

In [ ]:
# Now check if we do better on the tough conditions. 
# Hard parameters
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 7e3, 4e-5, 1, 0.0],
    tcr_nmf = [6, 4, 1],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct, psi0
    #car_fits = [3000.0, 10**(-3.5), 0.2, 0.02],
    car_fits = [1000.0, 1e-2, 10.0, 10.0, 0.0],
    car_k = 1,
    car_nmf = [3, 2, 3]  # Hard implicit CAR
)
# A log10 car fits causing RuntimeError issues
# 3.47425152 -3.56416617 -0.89229255 -1.85015833  1.09313421  1.15144979

l_range_better, tau_range_better, better_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_better, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)
l_range_brute, tau_range_brute, brute_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_brute, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

# If you see only one line of each color, it means both methods agree!
figaxes = plot_model_output(better_outs, l_range_better, tau_range_better, lstyle="-")
figaxes = plot_model_output(brute_outs, l_range_brute, tau_range_brute, lstyle="--", lw=3.0, figaxes=figaxes)
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

In [ ]:
# Test really tough conditions that makes the previous method fail
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 7e3, 4e-5, 1, 0.0],
    tcr_nmf = [6, 4, 1],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct, psi0
    car_fits = [3000.0, 10**(-3.5), 0.2, 0.02, 0.0],
    car_k = 1,
    car_nmf = [3, 2, 3]  # Hard implicit CAR
)
# A log10 car fits causing RuntimeError issues
# 3.47425152 -3.56416617 -0.89229255 -1.85015833  1.09313421  1.15144979

l_range_better, tau_range_better, better_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_better, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

figaxes = plot_model_output(better_outs, l_range_better, tau_range_better, lstyle="-")

try:
    l_range_brute, tau_range_brute, brute_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_brute, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)
except RuntimeError:
    print("The brute force method failed on this parameter set. ")
else:
    figaxes = plot_model_output(brute_outs, l_range_brute, tau_range_brute, lstyle="--", lw=3.0, figaxes=figaxes)
    
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

In [ ]:
# Other tough condition
# m, f of CAR = 3, 2
# log10 of parameters = 3.0342669  -3.41731933 -0.68163643 -0.20161508 
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 7e3, 4e-5, 1, 0.0],
    tcr_nmf = [6, 4, 1],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct
    car_fits = [1000.0, 10**(-3.4), 10**(-0.68), 10**(-0.2), 0.0],
    car_k = 2,
    car_nmf = [3, 3, 2]  # Hard implicit CAR
)

l_range_better, tau_range_better, better_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_better, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

figaxes = plot_model_output(better_outs, l_range_better, tau_range_better, lstyle="-")

try:
    l_range_brute, tau_range_brute, brute_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_brute, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)
except RuntimeError:
    print("The brute force method failed on this parameter set. ")
else:
    figaxes = plot_model_output(brute_outs, l_range_brute, tau_range_brute, lstyle="--", figaxes=figaxes)
    
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

# Compare to ODE integration
That's the only way to know for sure we are doing everything right. 

In [14]:
from scipy.integrate import solve_ivp
import itertools
import pandas as pd

In [15]:
def y_to_listvecs(y, ns, n_types):
    yvecs = []
    n_cumul = 0
    # Split y vector into receptor types for ease of use
    for i in range(n_types):
        # For each receptor type, there are N^i+1 complexes and one I^i
        n_vari_i = ns[i] + 2
        yvecs.append(y[n_cumul:n_cumul+n_vari_i])
        n_cumul += n_vari_i
    return yvecs

# Keep all I in a separate vector
def y_to_listvecs_final(y, ns, n_types):
    yvecs = []
    ivec = np.zeros(n_types)
    n_cumul = 0
    for i in range(n_types):
        n_vari_i = ns[i] + 2
        yvecs.append(y[n_cumul:n_cumul+n_vari_i-1])
        ivec[i] = y[n_cumul + n_vari_i-1]
        n_cumul += n_vari_i
    yvecs.append(ivec)
    return yvecs
        

def deriv_akpr_i_receptor_types(t, y, ratesp, tausp, lsp, ri_tots, nparams):
    # For use with solve_ivp
    # First compute separate lists of dy/dt, then concatenate into one vector
    phis, kappas, cmthreshs, ithreshs, ks, gamma_mat, psi0s  = ratesp
    Rsp, ITp = ri_tots
    ns, ms, fs = nparams
    n_types = len(Rsp)
    yvecs = y_to_listvecs(y, ns, n_types)
    # Also get the vector of I and of psi(I)
    ivec = np.asarray([y[-1] for y in yvecs])
    psivec = psi_of_i_gamma(ivec, ithreshs, ks, phis, gamma_mat, psi0s)
    cmvec = np.asarray([yvecs[i][ms[i]] for i in range(n_types)])
    
    # Now compute derivatives for each receptor type
    yderivs = []
    for i in range(n_types):
        n_vari_i = ns[i] + 2
        rbi = np.sum(yvecs[i][:n_vari_i-1])
        dy = np.zeros(n_vari_i)
        # First complex's forward rate. Assuming N > 0, else there is no KPR at all. 
        rho_0 = phis[i] if ns[i] - fs[i] > 0 else psivec[i]
        rho_prev = rho_0  # rho of C_{n-1}
        invtaui = 1.0 / tausp[i]
        dy[0] = kappas[i] * (lsp[i] - rbi) * (Rsp[i] - rbi) - (rho_0 + invtaui) * yvecs[i][0]
        for n in range(1, ns[i]):
            rho_n = phis[i] if n < ns[i] - fs[i] else psivec[i]
            dy[n] = rho_prev * yvecs[i][n-1] - (rho_n + invtaui) * yvecs[i][n]
            # Store rho_n into rho_prev for the next step
            rho_prev = rho_n
        # Last complex, no phosphorylation going further
        dy[ns[i]] = rho_prev * yvecs[i][ns[i]-1] - invtaui * yvecs[i][ns[i]]
        # Derivative of I^i. Missing the global rate beta_I, assume it is 1
        dy[ns[i]+1] = (ITp - np.sum(ivec)) * cmvec[i] / cmthreshs[i] - yvecs[i][ns[i]+1]
        yderivs.append(dy.copy())
    
    # Concatenate
    return np.concatenate(yderivs)

In [ ]:
def steady_akpr_i_receptor_types_ode(ratesp, tausp, lsp, ri_tots, nparams, rtol=1e-4):
    """Solving for the steady_state of the SHP-1 model with two
    different receptors, each with its own ligand, by integrating its 
    ODEs to stationarity. 

    Args:
        ratesp (list of np.ndarrays): [phi_arr, kappa_arr, cmthresh, sthresh_arr,
            k_arr, gamma_mat, psi_arr] where x_arr is
            a 1d array of parameter x values for each receptor type.
            except cmthresh which is unique,
            and gammat is a KxK array for K receptor types,
            containing \gamma_{ij}, contribution of SHP-1 bound to receptor type j
            to the negative feedforward on receptor type i.
        tausp (np.ndarray of floats): binding time of the ligands
            of each type of receptor
        lsp (np.ndarray of floats): total number of ligands of each type
        ri_tots (list of 1 ndarray, 1 float):
            Rsp (np.ndarray of floats): total number of receptors of each type
            ITp (float): total number of I molecules
        nparams (list of 3 np.ndarrays): Ns, ms, fs of all receptor types.

    Returns:
        complexes (np.ndarray): list of 1D arrays of complexes or S, ordered
            [ [C_0, C_1, ..., C_N],
              [D_0, D_1, ..., D_N'],
              [S_1, S_2]
            ]
    """
    # Integrate for a time typically long enough
    tblock = 400.0
    tspan = (0.0, tblock)
    t_eval = np.asarray([tblock-1.0, tblock])  # Need two final steps to check for convergence
    Rsp, Itot = ri_tots
    n_types = len(Rsp)
    total_n_variables = np.sum(nparams[0]) + 2*n_types  # 2*(N+2)
    y_init = np.zeros(total_n_variables)
    res = solve_ivp(deriv_akpr_i_receptor_types, tspan, y0=y_init, 
                    t_eval=t_eval, method="LSODA", 
                    args=(ratesp, tausp, lsp, ri_tots, nparams))
    
    # Check convergence
    try:
        rel_change = np.amax(np.abs((res.y[:, 1] - res.y[:, 0]) / res.y[:, 1]))
    except TypeError:
        print("Problem with convergence")
        print(res.y)
        if len(res.y) == 0:
            raise RuntimeError()
        else:
            rel_change = np.inf
    
    # If not reached yet, keep integrating
    while rel_change > rtol:
        tspan = (tspan[1], tspan[1]+tblock)
        t_eval = np.asarray([tspan[1]-1.0, tspan[1]])
        res = solve_ivp(deriv_akpr_i_receptor_types, tspan, y0=res.y[:, -1], 
                    t_eval=t_eval, method="LSODA", 
                        args=(ratesp, tausp, lsp, ri_tots, nparams))
        rel_change = np.amax(np.abs((res.y[:, 1] - res.y[:, 0]) / res.y[:, 1]))
        
    ysteady = res.y[:, -1]
    
    # Arrange into a list of vectors for each receptor type
    return y_to_listvecs_final(ysteady, nparams[0], n_types)

In [ ]:
# Check the tough case first
# Other tough condition
# m, f of CAR = 3, 2
# log10 of parameters = 3.0342669  -3.41731933 -0.68163643 -0.20161508 
# Easy parameters
all_rates, all_nmf = wrap_param_choices(
    # TCR fit: phi, kappa, cmthresh, sthresh, k_S, psi0
    tcr_params = [0.2, 1e-4, 7e3, 4e-5, 1, 0.0],
    tcr_nmf = [6, 4, 1],
    # CAR fit: cmthresh, sthresh, gamma_tc, gamma_ct
    car_fits = [1000.0, 10**(-3.4), 10**(-0.68), 10**(-0.2), 0.0],
    car_k = 2,
    car_nmf = [3, 3, 2]  # Hard implicit CAR
)


l_range_better, tau_range_better, better_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_better, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau)

figaxes = plot_model_output(better_outs, l_range_better, tau_range_better, lstyle="-")

try:
    l_range_ode, tau_range_ode, ode_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_ode, 
                rates=all_rates, ri_tots=tcr_car_ritots, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau, n_tau=101)
except RuntimeError:
    print("The ODE method failed on this parameter set. ")
else:
    figaxes = plot_model_output(ode_outs, l_range_ode, tau_range_ode, 
                                lstyle="--", figaxes=figaxes, lw=3.0)
    
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()

# More systematic check
Move that to a Python script and multiprocess across kmf choices. 

In [18]:
# Check random L, tau couples on a grid of N, m, f values for TCR and CAR
# Compare solution methods, make sure they agree for all N, m, f cases. 
# TCR nmf grid. Could multiprocess that in principle
def check_nkmf_ode_steady(nkmf_tcr, nkmf_car, tcr_fix, ri_tots, nreps=10, seed=None, rtol=1e-4):
    """ 
    Check if analytical solution and ODE integration agree for nreps
    random choices of model parameters and kmf_tcr, kmf_car
    tcr_fix: kappa
    """
    # return NaN if the grid point is impossible (basically, N < m or f)
    for nkmf in [nkmf_tcr, nkmf_car]:
        if nkmf[0] < nkmf[2]:
            return np.nan, []
        elif nkmf[0] < nkmf[3]:
            return np.nan, []
        
    rgen = np.random.default_rng(seed)
    # Choose bounds on fitted parameters. 
    # For TCR: fitting phi, cmthresh, ithresh, psi0
    tcr_fit_bounds = [(0.05, 1.0), (10.0, 10.0*ri_tots[0][0]), 
                      (1e-5*ri_tots[1], 10.0*ri_tots[1]), (1e-6, 1e-1)]
    tcr_bds = [np.log10(np.asarray(a)) for a in zip(*tcr_fit_bounds)]
    
    # For CAR: fitting cmthresh, ithresh, gamma_tc, gamma_ct, psi0
    car_fit_bounds = [(10.0, 10.0*ri_tots[0][1]), (1e-5*ri_tots[1], 10.0*ri_tots[1]), 
                     (1e-1, 1e0), (1e-1, 1e3), (1e-4, 1e-2)]
    car_bds = [np.log10(np.asarray(a)) for a in zip(*car_fit_bounds)]
    
    # Also random taus and Ls within reasonable ranges
    taus_fit_bounds = [(1.0, 10.0), (10.0, 80.0)]
    tau_bds = [np.asarray(a) for a in zip(*taus_fit_bounds)]
    l_fit_bounds = [(1e2, 1e5), (1e3, 1e6)]
    l_bds = [np.asarray(a) for a in zip(*l_fit_bounds)]
    
    
    # Sample parameters at random nreps times
    max_rel_differences = []
    strange_cases = []
    for k in range(nreps):
        # Sample TCR fit parameters
        tcr_samp = tcr_bds[0] + (tcr_bds[1]-tcr_bds[0])*rgen.random(size=len(tcr_bds[0]))
        tcr_samp = 10.0**tcr_samp
        # Sample CAR fit parameters
        car_samp = car_bds[0] + (car_bds[1]-car_bds[0])*rgen.random(size=len(car_bds[0]))
        car_samp = 10.0**car_samp
        # Sample taus and Ls
        tau_samp = tau_bds[0] + (tau_bds[1]-tau_bds[0])*rgen.random(size=len(tau_bds[0]))
        l_samp = l_bds[0] + (l_bds[1]-l_bds[0])*rgen.random(size=len(l_bds[0]))
        # Assemble all TCR rates needed for wrap_param_choices
        # TCR rates: phi, kappa, cmthresh, sthresh, k_S, psi0
        tcr_params = [tcr_samp[0], tcr_fix[0], *tcr_samp[1:3], nkmf_tcr[1], tcr_samp[3]]
        # Wrap rates
        all_rates, all_nmf = wrap_param_choices(
            tcr_params = tcr_params,
            tcr_nmf = [nkmf_tcr[0], *nkmf_tcr[2:]],
            car_fits = car_samp,
            car_k = nkmf_car[1],
            car_nmf = [nkmf_car[0], *nkmf_car[2:]]
        )
        # Compute steady-state with both methods and compare
        try:
            complexes1 = steady_akpr_i_receptor_types_better(all_rates, tau_samp, 
                                                             l_samp, ri_tots, all_nmf)
            complexes2 = steady_akpr_i_receptor_types_ode(all_rates, tau_samp, 
                                                    l_samp, ri_tots, all_nmf, rtol=rtol/2)
        except Exception as e:
            max_rel_differences.append(-1.0)
            print("Error for all_rates={}, all_nmf={}".format(all_rates, all_nmf), e)
        else:
            # Relative difference
            maxrel = np.amax(np.concatenate([np.abs((complexes1[i] - complexes2[i])/complexes1[i]) 
                          for i in range(len(complexes1))]))
            max_rel_differences.append(maxrel)
            if maxrel > rtol*100:
                strange_cases.append({
                    "nkmf_tcr": nkmf_tcr, 
                    "nkmf_car": nkmf_car, 
                    "tcr_fix": tcr_fix, 
                    "ri_tots": ri_tots, 
                    "all_rates": all_rates,
                    "tau_samp": tau_samp, 
                    "l_samp": l_samp, 
                    "all_nmf": all_nmf,
                    "max_rel_err": maxrel, 
                    "complexes_better": complexes1, 
                    "complexes_ode": complexes2
                })
    
    return max(max_rel_differences), strange_cases

In [ ]:
# Grid over kmf of TCR and CAR. 
# This takes a minute or so to run. 
nkmf_bounds_tcr = [(4, 6), (1, 2), (1, 5), (1, 2)]
grid_tcr = list(itertools.product(*[range(a, b+1) for a, b in nkmf_bounds_tcr]))
# CAR nmf grid
nkmf_bounds_car = [(1, 3), (1, 2), (1, 3), (1, 3)]
grid_car = list(itertools.product(*[range(a, b+1) for a, b in nkmf_bounds_car]))
tcr_fixr = [1e-4]  # kappa

# Test each combination of TCR and CAR nmf grid points. 
# Save the max relative difference for each pair of kmf choices
df_maxdiffs = pd.DataFrame(
        np.zeros([len(grid_tcr), len(grid_car)]), 
        index=pd.MultiIndex.from_tuples(grid_tcr, names=["N", "k", "m", "f"]), 
        columns=pd.MultiIndex.from_tuples(grid_car, names=["N", "k", "m", "f"])
    )
seed_seq = np.random.SeedSequence(entropy=0x31ecd927098b4760c2d043c9342a66ba, 
                                  spawn_key=(0x710eec56d9ef5eb8f15382fa53a7ffed,))
seeds = seed_seq.spawn(len(grid_tcr)*len(grid_car))
higherr_cases = []
for nkmf_t in grid_tcr:
    for nkmf_c in grid_car:
        sd = seeds.pop()
        maxdiff, stranges = check_nkmf_ode_steady(nkmf_t, nkmf_c, tcr_fixr, 
                                    tcr_car_ritots, nreps=1, seed=sd, rtol=1e-4)
        df_maxdiffs.loc[nkmf_t, nkmf_c] = maxdiff
        higherr_cases += stranges
# Keep only feasible values
df_maxdiffs = df_maxdiffs.dropna(axis=0, how="all").dropna(axis=1, how="all")

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(df_maxdiffs)
fig.colorbar(im, label="Max. relative error")
fig.set_size_inches(6, 8)
ax.set_xticks(np.arange(df_maxdiffs.shape[1]))
ax.set_xticklabels(df_maxdiffs.columns, rotation=90, fontsize=8)
ax.set_yticks(np.arange(df_maxdiffs.shape[0]))
ax.set_yticklabels(df_maxdiffs.index, fontsize=8)
ax.grid(color='grey', linestyle='-', linewidth=0.5, alpha=0.5)
ax.set(xlabel="CAR (n, k, m, f)", ylabel="TCR (n, k, m, f)")
plt.show()
plt.close()

In [21]:
# Double-check high error cases
seeds_stranges = np.random.SeedSequence(entropy=0x90460de90cb4a64088b41149e1a5924c, 
                    spawn_key=(0x78ab3d1e4c174c350276e5fd38f75830,)).spawn(len(higherr_cases))
for stranges in higherr_cases:
    print("High error, {a[max_rel_err]}, in {a[nkmf_tcr]}, {a[nkmf_car]}".format(a=stranges))
    print("Parameters sampled then: {a[all_rates]}".format(a=stranges))
    #print("For Ls = {a[l_samp]} and taus = {a[tau_samp]}".format(a=stranges))
    print("ODE-based solution at SS: {a[complexes_ode]}".format(a=stranges))
    print("Analytical solution at SS: {a[complexes_better]}".format(a=stranges))
    # Re-test that condition
    re_test_condition, _ = check_nkmf_ode_steady(*max_diff_arg, tcr_fixr, 
                tcr_car_ritots, nreps=20, seed=seeds_stranges.pop(), rtol=1e-6)
    # After re-test:
    print("After re-test, diff is now :", re_test_condition)
    if re_test_condition < 1e-2:
        print("This is fine, it was just a strange choice of parameter samples and incomplete convergence")
    else:
        print("This case warrants further investigation")
    print()

In [ ]:
# Investigate the case where there seems to be bimodality for the CAR, if any
if len(higherr_cases) > 0:
    stranges = higherr_cases[0]
    n_variables = np.sum(stranges['all_nmf'][0]) + 4
    t_eval = np.arange(400.0)
    soln = solve_ivp(deriv_akpr_i_receptor_types, (0.0, 400.0), y0=np.zeros(n_variables), 
                        t_eval=t_eval, method="LSODA", 
                        args=(stranges['all_rates'], stranges['tau_samp'], 
                            stranges['l_samp'], stranges['ri_tots'], stranges['all_nmf']))

    # Try to initialize it at the stationary steady-state, see if stable
    yinit_2 = steady_akpr_i_receptor_types_better(stranges['all_rates'], stranges['tau_samp'], 
                            stranges['l_samp'], stranges['ri_tots'], stranges['all_nmf'])
    yinit_2 = np.concatenate([yinit_2[0], [yinit_2[2][0]], yinit_2[1], [yinit_2[2][1]]])
    soln_2 = solve_ivp(deriv_akpr_i_receptor_types, (0.0, 400.0), y0=yinit_2, 
                        t_eval=t_eval, method="LSODA", 
                        args=(stranges['all_rates'], stranges['tau_samp'], 
                            stranges['l_samp'], stranges['ri_tots'], stranges['all_nmf']))

    fig, axes = plt.subplots(1, 2, sharey=True)
    labels = [r"$C^T_{}$".format(i) for i in range(stranges['all_nmf'][0][0]+1)] + [r"$I^T$"]
    labels += [r"$C^C_{}$".format(i) for i in range(stranges['all_nmf'][0][1]+1)] + [r"$I^C$"]
    for i in range(stranges['all_nmf'][0][0]+1, soln.y.shape[0]):
        axes[0].plot(soln.t, soln.y[i], label=labels[i])
        axes[1].plot(soln_2.t, soln_2.y[i], label=labels[i])
    for ax in axes.flatten():
        ax.set(xlabel="Time (s)", ylabel="Complex", yscale="log")
    axes.flat[0].set_title("Initialized at 0")
    axes.flat[1].set_title("Initialized at analytical SS")
    ax.legend()
    plt.show()
    plt.close()
else:
    print("No case where the error is high but finite, suggesting there is no bimodality we could have forgotten")
    print("Cases of infinite error are cases where the numerical evaluation of the steady-state solution fails")
    print("because of low tolerance/limited number of iterations, etc.")

There are clearly cases where the TCR or CAR are bimodal or more, such as the example above, where we see that the analytical SS is stable, but has higher $C^C_n$ affected by the inhibitory feedback than in the ODE solution, which is also stable. 

However, they do not seem to arise ofen, so we can disregard during MCMC for simplicity. 

## Cases that cause issues during MCMC simulations
Caught some exceptions -- discrepancies between final computed and solution $C_m$ when one receptor type is implicit -- and their associated parameters, need to check why they occur. Could just be because of extreme parameter values causing truncation, overflow, or cancellation errors. There are generally for completely crazy large/small parameter values, but here we just try to check if we can reproduce these errors with the reporter parameter values. 

Examples below:

```
Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [1.e-03 5.e+02] 
lsp: [   530.26995012 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]
Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [  0.5 500. ] 
lsp: [   530.26995012 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]
Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [  1.76829702 500.        ] 
lsp: [   530.26995012 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]
Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [  2.89429038 500.        ] 
lsp: [   530.26995012 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]

Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [  4.7134657 500.       ] 
lsp: [   530.26995012 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]

Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [  5.99314161 500.        ] 
lsp: [   530.26995012 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]



Difference in implicit solution and final calculated C_m: 119535.90819243775 0.4120087479671796 for parameter values as follows:
ratesp: [array([0.04312739, 0.00021564]), array([0.0001, 0.001 ]), array([4.07960031e+002, 6.64940561e-304]), array([4.98476903e-04, 5.06936911e-80]), array([1, 2]), array([[1.00000000e+000, 2.00759969e+220],
       [1.52301329e-308, 1.00000000e+000]]), array([9.68027379e-06, 3.18963039e-05])] 
tausp: [  5.99314161 500.        ] 
lsp: [ 52312.36226793 106510.14384853] 
ri_tots: [array([121793.30786718, 981184.8823273 ]), 1.0] 
nparams: [array([6, 3]), array([3, 3]), array([1, 3])]
```

In [24]:
from models.tcr_car_akpr_model import steady_akpr_i_receptor_types

In [ ]:
# Copy-pasted one example from above. 
# All things considered, some parameters are really extreme 
# (gamma^T_C, gamma^C_T at 1e+220, 1e-308, C_{m, th} at 1e-304, I^C_th at 1e-80)
# so it is pleasantly surprising that the numerical solution
# fails only for such crazy examples. 
# MCMC goes there seldomly because parameters are scanned in log10 scale, 
# so log10(1e-308) is not that far from log10(1.0), really. 
# And notice this is N=3, m=3, f=3, k_I=2 for the CAR, which is a 
# strongly non-linear, deep inhibitory feedback in the KPR cascade

# I can't actually reproduce the issue with root_scalar that I got from the computational cluster, 
# this must be a platform-specific issue with very small/large floating point numbers. 
# Anyways, nothing wrong with the solution method, just crazy parameter values, it comes out all right here. 
# Need a way to limit further the TCR/CAR parameters in MCMC simulations. 
all_rates = [
    np.asarray([0.04312739, 0.00021564]),   # phis
    np.asarray([0.0001, 0.001 ]),   # kappas
    np.asarray([4.07960031e+002, 6.64940561e-304]),   # C_m,th
    np.asarray([4.98476903e-04, 5.06936911e-80]),   # I_th
    np.asarray([1, 2]), # k_I
    np.asarray([[1.00000000e+000, 2.00759969e+220],
               [1.52301329e-308, 1.00000000e+000]]), 
    np.asarray([9.68027379e-06, 3.18963039e-05])
]
all_nmf = [np.asarray([6, 3]), np.asarray([3, 3]), np.asarray([1, 3])]
tcr_car_ritots2 = [np.asarray([121793.30786718, 981184.8823273 ]), 1.0]
choice_cd19_l_tau2 = (106510.14384853, 500.0)

l_range_better, tau_range_better, better_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types,#_better, 
                rates=all_rates, ri_tots=tcr_car_ritots2, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau2, 
                tcr_l_range=np.asarray([530.26995012, 5000.0, 52312.36226793]), 
                tcr_tau_range=np.linspace(5.99314161, 10.01, 11))

figaxes = plot_model_output(better_outs, l_range_better, tau_range_better, lstyle="-")
print("Finished computing the steady-state solution")
try:
    l_range_ode, tau_range_ode, ode_outs = check_model_output_tcr_car(
                model=steady_akpr_i_receptor_types_ode, 
                rates=all_rates, ri_tots=tcr_car_ritots2, 
                nparams=all_nmf, cd19_l_tau=choice_cd19_l_tau2, n_tau=101, 
                tcr_l_range=np.asarray([530.26995012, 5000.0, 50000.0]), 
                tcr_tau_range=np.linspace(0.500, 10.01, 11))
except RuntimeError:
    print("The ODE method failed on this parameter set. ")
else:
    figaxes = plot_model_output(ode_outs, l_range_ode, tau_range_ode, 
                                lstyle="--", figaxes=figaxes, lw=3.0)
    
axes = figaxes[1]
axes[0].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$T_{N_T}$")
axes[1].set(xlabel=r"$\tau_{TCR}$", ylabel=r"$C_{N_C}$")
for ax in axes:
    ax.set(yscale="log")
figaxes[0].tight_layout()
plt.show()
plt.close()